In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import warnings
from blackscholes import Black76Call
import plotly.express as px

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def get_data():
    resp = requests.get(
                
                'http://192.168.0.172:8080/api/v1/data/settlement?symbols=MESc1&from=2023-01-01&to=2024-04-17')
        
    data = resp.json()
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.strftime("%m/%d/%Y")
    df['timestamp'] = pd.to_datetime(df['timestamp'])


    loc_index = df.columns.get_loc('timestamp') + 1
    df.insert(loc=loc_index,column='Month',value=df['timestamp'].dt.month)
    loc_index = df.columns.get_loc('timestamp') + 2
    df.insert(loc=loc_index,column='Year',value=df['timestamp'].dt.year)
    return df

In [4]:
import numpy as np
from scipy.stats import norm
N = norm.cdf


def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = Black76Call(S, K, T, r, sigma).price()
        vega = Black76Call(S, K, T, r, sigma).vega()
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

def implied_volatility_bs(option_price, S, K,  T ,r, option_type='call'):
    tolerance = 0.0001
    lower_volatility = 0.01  # starting lower bound for volatility
    upper_volatility = 4.0   # starting upper bound for volatility

    while True:
        mid_volatility = (lower_volatility + upper_volatility) / 2
        theoretical_price = Black76Call(S , K, T, r, mid_volatility).price()
        
        if abs(theoretical_price - option_price) < tolerance:
            return mid_volatility
        
        if theoretical_price < option_price:
            lower_volatility = mid_volatility
        else:
            upper_volatility = mid_volatility

In [5]:
def round( n ): 
  
    # Smaller multiple 
    a = (n // 10) * 10
      
    # Larger multiple 
    b = a + 10
      
    # Return of closest of two 
    return (b if n - a > b - n else a) 


month_code = dict( {
    1: 'F',
    2: 'G',
    3: 'H',
    4: 'J',
    5: 'K',
    6: 'M',
    7: 'N',
    8: 'Q',
    9: 'U',
    10: 'V',
    11: 'X',
    12: 'Z'
})

def last_two_digit_year(year):
    return year % 100

def get_symbol(df):
    df['call_symbol'] = ("@"+'EW'+df['month_code']+df['last_digit_year']+"C"+df['strike_price']+df['hundred']).astype(str)   
    df['put_symbol'] = ("@"+'EW'+df['month_code']+df['last_digit_year']+"P"+df['strike_price']+df['hundred']).astype(str)
    return df   
def myround(x, base=10):
    return base * round(x/base) 

In [6]:
def df_pre_process(df):
    df['strike_price'] = df['settlement'].apply(lambda x: round(x))
    df['strike_price'] = df['strike_price'].astype(int)
    df['month_code'] = df['Month'].map(month_code).astype(str)
    df['last_digit_year'] = df['Year'].apply(lambda x: last_two_digit_year(x)).astype(str)
    df['last_digit_year'] = df['last_digit_year'].astype(str)
    df['strike_price'] = (df['strike_price']).astype(str)
    df['hundred'] = '00'
    df = get_symbol(df)
    df = df[['timestamp' , 'call_symbol', 'put_symbol' , 'strike_price']]
    df['start_time'] = df['timestamp']
    df['end_time'] = df['timestamp'].shift(-1)
    return df

In [7]:
df1 = get_data()

In [8]:
df = df1.copy()
df = df_pre_process(df).dropna()

C:\Users\IBISP\AppData\Local\Temp\ipykernel_28064\976365482.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_time'] = df['timestamp']
C:\Users\IBISP\AppData\Local\Temp\ipykernel_28064\976365482.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end_time'] = df['timestamp'].shift(-1)


In [9]:
df_call = pd.read_csv(r'C:\Users\IBISP\Desktop\Bhargav\Option Chain\S&P_ATMData_03_01_23_to_16_04_24_Call.csv')

In [10]:
df_expire =  df.groupby([ df['timestamp'].dt.year , df['timestamp'].dt.month]).agg({'timestamp':'last'})

In [11]:
df_expire['month'] = df_expire['timestamp'].dt.month
df_expire['year'] = df_expire['timestamp'].dt.year
df_expire.columns = ['timestamp' , 'month' , 'year']

In [12]:
df_expire

timestamp  month  year
timestamp timestamp                        
2023      1         2023-01-31      1  2023
          2         2023-02-28      2  2023
          3         2023-03-31      3  2023
          4         2023-04-28      4  2023
          5         2023-05-31      5  2023
          6         2023-06-30      6  2023
          7         2023-07-31      7  2023
          8         2023-08-31      8  2023
          9         2023-09-29      9  2023
          10        2023-10-31     10  2023
          11        2023-11-30     11  2023
          12        2023-12-29     12  2023
2024      1         2024-01-31      1  2024
          2         2024-02-29      2  2024
          3         2024-03-28      3  2024
          4         2024-04-16      4  2024

In [13]:
df1

,timestamp,Month,Year,sym,settlement
0,2023-01-03,1,2023,MESc1,3846.00
1,2023-01-04,1,2023,MESc1,3874.50
2,2023-01-05,1,2023,MESc1,3829.00
3,2023-01-06,1,2023,MESc1,3915.50
4,2023-01-09,1,2023,MESc1,3913.75
...,...,...,...,...,...
320,2024-04-11,4,2024,MESc1,5243.25
321,2024-04-12,4,2024,MESc1,5167.50
322,2024-04-15,4,2024,MESc1,5104.00
323,2024-04-16,4,2024,MESc1,5092.50


In [15]:
df_call['date'] = pd.to_datetime(df_call['date'])
df_call['date'] = df_call['date'].dt.strftime("%m/%d/%Y")
df_call['date'] = pd.to_datetime(df_call['date'])
df_call['timestamp'] = df_call['date']
df_call['month'] = df_call['timestamp'].dt.month
df_call['year'] = df_call['timestamp'].dt.year

df_call = pd.merge(df_call , df1[['timestamp' , 'settlement']] , on='timestamp')


df_call = df_call[['date', 'open', 'high', 'low', 'close', 'volume',
       'open_int', 'timestamp','month', 'year', 'settlement']]

df_call = pd.merge(df_call , df_expire , on=['month' , 'year'])
df_call['dte'] = (df_call['timestamp_y'] - df_call['date']).dt.days



In [16]:
df_call['strike_price'] = df_call['settlement'].apply(lambda x: round(x))
df_call = df_call[df_call['dte'] != 0]

In [28]:
df_call.tail(60)

,date,open,high,low,close,volume,open_int,timestamp_x,month,year,settlement,timestamp_y,dte,strike_price,atm_volatility,atm_volatility_bs
260,2024-01-16,51.25,51.25,36.25,42.75,1025,2672,2024-01-16,1,2024,4798.50,2024-01-31,15,4800.0,0.112316,0.112316
261,2024-01-17,44.25,44.75,36.75,44.75,142,887,2024-01-17,1,2024,4771.25,2024-01-31,14,4770.0,0.118630,0.118630
262,2024-01-18,25.75,43.25,25.00,42.00,240,670,2024-01-18,1,2024,4811.25,2024-01-31,13,4810.0,0.114457,0.114457
263,2024-01-19,21.50,39.25,17.75,37.00,374,903,2024-01-19,1,2024,4869.50,2024-01-31,12,4870.0,0.105937,0.105937
264,2024-01-22,37.25,46.00,33.00,33.75,660,984,2024-01-22,1,2024,4881.00,2024-01-31,9,4880.0,0.108897,0.108897
265,2024-01-23,30.25,35.00,24.50,32.25,270,2131,2024-01-23,1,2024,4895.00,2024-01-31,8,4890.0,0.102859,0.102859
266,2024-01-24,32.50,48.50,24.75,28.25,948,4169,2024-01-24,1,2024,4898.00,2024-01-31,7,4900.0,0.108141,0.108141
267,2024-01-25,20.25,32.00,20.25,30.25,161,1688,2024-01-25,1,2024,4923.25,2024-01-31,6,4920.0,0.113703,0.113703
268,2024-01-26,25.75,33.50,19.75,22.50,952,1864,2024-01-26,1,2024,4916.25,2024-01-31,5,4920.0,0.106019,0.106018
269,2024-01-29,8.30,26.00,8.20,24.25,1353,3292,2024-01-29,1,2024,4954.50,2024-01-31,2,4950.0,0.149981,0.149980


In [30]:
df

,timestamp,call_symbol,put_symbol,strike_price,start_time,end_time
0,2023-01-03,@EWF23C385000,@EWF23P385000,3850,2023-01-03,2023-01-04
1,2023-01-04,@EWF23C387000,@EWF23P387000,3870,2023-01-04,2023-01-05
2,2023-01-05,@EWF23C383000,@EWF23P383000,3830,2023-01-05,2023-01-06
3,2023-01-06,@EWF23C392000,@EWF23P392000,3920,2023-01-06,2023-01-09
4,2023-01-09,@EWF23C391000,@EWF23P391000,3910,2023-01-09,2023-01-10
...,...,...,...,...,...,...
319,2024-04-10,@EWJ24C521000,@EWJ24P521000,5210,2024-04-10,2024-04-11
320,2024-04-11,@EWJ24C524000,@EWJ24P524000,5240,2024-04-11,2024-04-12
321,2024-04-12,@EWJ24C517000,@EWJ24P517000,5170,2024-04-12,2024-04-15
322,2024-04-15,@EWJ24C510000,@EWJ24P510000,5100,2024-04-15,2024-04-16


In [18]:
df_call['atm_volatility'] = df_call.apply(lambda x: find_vol(x['close'] , x['settlement'] , x['strike_price'] , x['dte']/365 , 0.055) , axis=1)

In [19]:
df_call['atm_volatility_bs'] = df_call.apply(lambda x: implied_volatility_bs(x['close'] , x['settlement'] , x['strike_price'] , x['dte']/365 , 0.055) , axis=1)

In [20]:
def strike_price_76(F ,sigma ,  T ,delta, option_type = 'call' ):
    #  T option dte
    #  t time to future expiry
    if option_type == 'call':
 
        result = (F* np.exp( - norm.ppf(delta ) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2))  
        return myround(result) 
    else:
        delta =  delta + 1

        result = F* np.exp(  - norm.ppf(delta ) * sigma * np.sqrt(T) -(sigma ** 2 * T) / 2) 
        return myround(result)
    

In [21]:
Target_delta = 0.25

In [22]:
def first_strike(F ,r ,sigma ,  T  ,Targeted_delta, option_type = 'call' ):
    new_strike = strike_price_76(F  ,sigma ,  T  ,Targeted_delta, option_type = 'call' )

    # replace option price with fetching from new data
    option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2

    strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
    derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
    
    return new_strike , derived_delta , strike_volatility



In [23]:
# def strike_for_target_delta(F ,r ,sigma ,  T , t ,Targeted_delta, derived_delta, option_type = 'call' ):
#     new_strike , derived_delta , strike_volatility = first_strike(F ,r ,sigma ,  T , t ,Targeted_delta, option_type = 'call' )
#     i = 1
#     old_delta = derived_delta 
#     while i < 15:
#         if abs(round(Targeted_delta,2) - round(derived_delta,2)) < 0.001:
#             # if (Targeted_delta - old_delta) < (Targeted_delta - derived_delta):
#             #     print(f"Strike price for ad {old_delta} delta:", old_strike)
#             # else:
#             if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.00000001:
#                 # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike)
#                 return new_strike , derived_delta
#             if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.00001:
#                 if Targeted_delta > 80:
#                     # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike - 10)
#                     return new_strike - 10 , derived_delta
#                 elif Targeted_delta < 20:
#                     # print(f"Strike price for {Targeted_delta} Targeted_delta:", new_strike + 10)
#                     return new_strike + 10 , derived_delta
#             break
#         else:
#             if Targeted_delta < derived_delta:

#                 old_strike = new_strike
#                 new_strike = old_strike + 10
#                 old_delta = derived_delta

#                 option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
#                 strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
#                 # derived_delta = mibian.BS([F, new_strike, r, T*365] , volatility= strike_volatility * 100)
#                 derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
#                 # print(f"Actual Delta: {derived_delta}")

#             else:

#                 old_strike = new_strike
#                 new_strike = old_strike - 10
#                 old_delta = derived_delta

#                 option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
#                 strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
#                 derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
#             # print(" Strike price = " , new_strike, "Delta = ", derived_delta)
#             return new_strike , derived_delta
#         i += 1  
#     if i == 15:
#         if Targeted_delta > 0.7:
#             # print("Strike price  = " , min(new_strike ,old_strike), "Delta = ", derived_delta)
#             return min(new_strike ,old_strike) , derived_delta
#         else:
#             # print("Strike price  = " , max(new_strike ,old_strike), "Delta = ", derived_delta)
#             return max(new_strike ,old_strike) , derived_delta

In [24]:
def strike_price_76(F  ,sigma ,  T  ,delta, option_type = 'call' ):
    #  T option dte
    #  t time to future expiry
    if option_type == 'call':
        # result = (F* np.exp( - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2)) 
        result = (F* np.exp( - norm.ppf(delta ) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2))  # with less error
        return myround(result) 
    else:
        delta =  delta + 1
        # result = F* np.exp(  - norm.ppf(delta * np.exp(r*t)) * sigma * np.sqrt(T) - (sigma ** 2 * T) / 2) 
        result = F* np.exp(  - norm.ppf(delta ) * sigma * np.sqrt(T) -(sigma ** 2 * T) / 2) # with less error
        return myround(result)
    

def first_strike(F ,r ,sigma ,  T  ,Targeted_delta ):
    new_strike = strike_price_76(F  ,sigma ,  T ,Targeted_delta, option_type = 'call' )

    # replace option price with fetching from new data
    option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2

    strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
    derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
    
    return new_strike , derived_delta , strike_volatility

def strike_for_target_delta(df , F ,r ,sigma ,  T , t ,Targeted_delta):
    new_strike , derived_delta , strike_volatility = first_strike(F ,r ,sigma ,  T  ,Targeted_delta)
    i = 1
    old_delta = derived_delta 
    while i < 15:
        if abs(round(Targeted_delta,4) - round(derived_delta,4)) < 0.001:
            
            if abs( round(Targeted_delta,4) - round(derived_delta,4)) < 0.0001:
                return new_strike , derived_delta
                # print(f"Strike price1 for {Targeted_delta} Targeted_delta:", new_strike)
            else:
                if Targeted_delta > 0.80:
                    return new_strike - 10 , derived_delta
                    # print(f"Strike price2 for {Targeted_delta} Targeted_delta:", new_strike - 10)
                elif Targeted_delta < 0.20:
                    return new_strike + 10 , derived_delta
                    # print(f"Strike price3 for {Targeted_delta} Targeted_delta:", new_strike + 10)
            
        else:
            if Targeted_delta < derived_delta:

                old_strike = new_strike
                new_strike = old_strike + 10
                old_delta = derived_delta

                option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
                strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
                # derived_delta = mibian.BS([F, new_strike, r, T*365] , volatility= strike_volatility * 100)
                derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
                # print(f"Actual Delta: {derived_delta}")

            else:

                old_strike = new_strike
                new_strike = old_strike - 10
                old_delta = derived_delta

                option_price = (df.loc[df['Strike'] == new_strike]['BidPrice'].values[0] + df.loc[df['Strike'] == new_strike]['OfferPrice'].values[0])/2
                strike_volatility = find_vol(option_price, F, new_strike,  T ,r)
                derived_delta = Black76Call(F, new_strike,T, r, strike_volatility).delta()
            # print(" Strike price1 = " , new_strike, "Delta = ", derived_delta)
        i += 1  
    if i == 15:
        if Targeted_delta > 0.7:
            # print("Strike price  = " , min(new_strike ,old_strike), "Delta = ", derived_delta)
            return min(new_strike ,old_strike) , derived_delta
        elif Targeted_delta < 0.3:
            # print("Strike price  = " , max(new_strike ,old_strike), "Delta = ", derived_delta)
            return max(new_strike ,old_strike) , derived_delta
    return new_strike , derived_delta

In [26]:
df_call

,date,open,high,low,close,volume,open_int,timestamp_x,month,year,settlement,timestamp_y,dte,strike_price,atm_volatility,atm_volatility_bs
0,2023-01-03,116.00,116.00,76.50,84.75,237,490,2023-01-03,1,2023,3846.00,2023-01-31,28,3850.0,0.204865,0.204865
1,2023-01-04,82.75,96.50,73.25,84.75,231,512,2023-01-04,1,2023,3874.50,2023-01-31,27,3870.0,0.197156,0.197156
2,2023-01-05,82.50,84.00,81.75,81.75,211,213,2023-01-05,1,2023,3829.00,2023-01-31,26,3830.0,0.202527,0.202527
3,2023-01-06,52.25,78.25,52.25,73.00,333,418,2023-01-06,1,2023,3915.50,2023-01-31,25,3920.0,0.184605,0.184605
4,2023-01-09,81.00,98.00,76.25,76.25,100,507,2023-01-09,1,2023,3913.75,2023-01-31,22,3910.0,0.194759,0.194759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2024-04-09,61.50,69.50,46.25,62.25,135,703,2024-04-09,4,2024,5260.25,2024-04-16,7,5260.0,0.214009,0.214009
319,2024-04-10,60.50,63.50,55.00,62.50,180,296,2024-04-10,4,2024,5207.75,2024-04-16,6,5210.0,0.239003,0.239003
320,2024-04-11,58.75,68.00,58.75,60.50,66,513,2024-04-11,4,2024,5243.25,2024-04-16,5,5240.0,0.240692,0.240692
321,2024-04-12,90.00,90.00,64.50,64.50,172,217,2024-04-12,4,2024,5167.50,2024-04-16,4,5170.0,0.304749,0.304749
